In [1]:
import os
from dotenv import load_dotenv
from prompt2map import Prompt2Map
from prompt2map.application.prompt2sql.sql_query_processor import prettify_sql


load_dotenv()

True

# Using a file

In [10]:
p2m = Prompt2Map.from_file(
    "censo2021portugal", 
    "../data/censo_pt_2021/geodata.parquet",
    "../data/censo_pt_2021/embeddings.parquet",
    "../data/censo_pt_2021/variable_descriptions.csv")

query = "Population density map of the district of Setúbal by parish in inhabitants / km2"
m = p2m.to_map(query)
m

In [11]:
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('image.png')

The geckodriver version (0.34.0) detected in PATH at /opt/homebrew/bin/geckodriver might not be compatible with the detected firefox version (129.0.2); currently, geckodriver 0.35.0 is recommended for firefox 129.*, so it is advised to delete the driver in PATH and retry


In [12]:
print(prettify_sql(p2m.retriever.sql_query))

SELECT Freguesia,
       Municipio,
       SUM(N_INDIVIDUOS) / (SUM(shape_area_m2) / 1000000) AS population_density_per_km2,
       ST_Union_Agg(geometry) AS geometry
FROM censo2021portugal
WHERE Distrito = 'Setúbal'
GROUP BY Freguesia,
         Municipio


# Using a PostGIS database

In [4]:
db_name = os.environ.get("DB_NAME")
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")
geo_table = "comuna"
geo_column = "geom"

p2m = Prompt2Map.from_postgis(geo_table, geo_column, db_name, db_user, db_password)
prompt = "Resultados del Partido Socialista en elección de diputados 2017, comunas de Paredones, Lolol, Santa Cruz"
m = p2m.to_map(prompt)
m

/Users/jm/Code/prompt2map/.conda/lib/python3.12/site-packages/geopandas/io/sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
/Users/jm/Code/prompt2map/.conda/lib/python3.12/site-packages/geopandas/io/sql.py:473: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(spatial_ref_sys_sql, con)


In [5]:
print(prettify_sql(p2m.retriever.sql_query))

SELECT comuna.nombre AS comuna_nombre,
       SUM(votos_mesa.votos) AS total_votos_socialista,
       ST_Union(geom) AS geom
FROM votos_mesa
JOIN mesa ON votos_mesa.mesa_id = mesa.id
JOIN comuna ON mesa.comuna_id = comuna.id
JOIN candidatura ON votos_mesa.candidatura_id = candidatura.id
JOIN partido ON candidatura.partido_id = partido.id
JOIN eleccion ON candidatura.eleccion_id = eleccion.id
WHERE eleccion.nombre = 'Diputados 2017'
  AND partido.nombre = 'Partido Socialista De Chile'
  AND comuna.nombre IN ('Paredones',
                        'Lolol',
                        'Santa Cruz')
GROUP BY comuna.nombre
